In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_dir = '/content/drive/MyDrive/Wheat Disease Dataset'

In [ ]:
import os

data_dir = '/content/drive/MyDrive/Wheat Disease Dataset'
print("Subfolders inside data_dir:", os.listdir(data_dir))

Subfolders inside data_dir: ['YellowRust', 'BrownRust', 'Mildew', 'Septoria', 'Healthy']


In [ ]:
import os
import shutil
import random
from tqdm import tqdm

original_data_dir = "/content/drive/MyDrive/Wheat Disease Dataset"
base_dir = "/content/drive/MyDrive/wheat-disease-split"  # new folder

# Create train/valid/test directories
for split in ['train', 'valid', 'test']:
    for cls in os.listdir(original_data_dir):
        os.makedirs(os.path.join(base_dir, split, cls), exist_ok=True)

# Move images into respective folders
for cls in tqdm(os.listdir(original_data_dir)):
    cls_path = os.path.join(original_data_dir, cls)
    if not os.path.isdir(cls_path):
        continue

    images = os.listdir(cls_path)
    random.shuffle(images)

    total = len(images)
    train_split = int(0.7 * total)
    valid_split = int(0.85 * total)

    for i, img in enumerate(images):
        src = os.path.join(cls_path, img)
        if i < train_split:
            dst = os.path.join(base_dir, 'train', cls, img)
        elif i < valid_split:
            dst = os.path.join(base_dir, 'valid', cls, img)
        else:
            dst = os.path.join(base_dir, 'test', cls, img)
        shutil.copy2(src, dst)

100%|██████████| 5/5 [05:17<00:00, 63.56s/it]


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths and image size
data_dir = "/content/drive/MyDrive/wheat-disease-split"
IMG_SIZE = 224
BATCH_SIZE = 32

# Augmentation for training, just rescaling for validation and test
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

# Load images from folders
train_generator = train_datagen.flow_from_directory(
    os.path.join(data_dir, 'train'),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

valid_generator = val_test_datagen.flow_from_directory(
    os.path.join(data_dir, 'valid'),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    os.path.join(data_dir, 'test'),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=1,
    class_mode='categorical',
    shuffle=False
)

Found 994 images belonging to 5 classes.
Found 559 images belonging to 5 classes.
Found 566 images belonging to 5 classes.


In [ ]:
!pip install efficientnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.3 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import efficientnet.tfkeras as efn
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Load EfficientNetB0 base model
base_model = efn.EfficientNetB0(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # Freeze base layers

# Build final model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(train_generator.num_classes, activation='softmax')
])

# Compile model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

16804768/16804768 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=10  # You can change this to 15–20 for better results
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 342s 10s/step - accuracy: 0.2013 - loss: 1.8552 - val_accuracy: 0.4061 - val_loss: 1.4249
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 164s 5s/step - accuracy: 0.3097 - loss: 1.5665 - val_accuracy: 0.4741 - val_loss: 1.3314
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 171s 5s/step - accuracy: 0.4237 - loss: 1.4064 - val_accuracy: 0.5206 - val_loss: 1.2555
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 192s 6s/step - accuracy: 0.4086 - loss: 1.4236 - val_accuracy: 0.5546 - val_loss: 1.2037
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 167s 5s/step - accuracy: 0.4593 - loss: 1.3397 - val_accuracy: 0.6029 - val_loss: 1.1428
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 170s 5s/step - accuracy: 0.4499 - loss: 1.3096 - val_accuracy: 0.6279 - val_loss: 1.1050
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 194s 6s/step - accuracy: 0.5009 - loss: 1.2517 - val_accuracy: 0.6512 - val_loss: 1.0559
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 191s 6s/step - accuracy: 0.5069 - loss: 1.2160 - val_accuracy: 0.6547 - 

In [ ]:
# Evaluate model on test data
test_loss, test_acc = model.evaluate(test_generator)
print(f"\n🔍 Test Accuracy: {test_acc*100:.2f}%")

566/566 ━━━━━━━━━━━━━━━━━━━━ 126s 223ms/step - accuracy: 0.6573 - loss: 0.9832

🔍 Test Accuracy: 65.19%


In [ ]:
# Unfreeze EfficientNet for fine-tuning
base_model.trainable = True

# Freeze all but the last 20 layers
for layer in base_model.layers[:-20]:
    layer.trainable = False

print(f"Trainable layers: {sum([l.trainable for l in base_model.layers])}")

Trainable layers: 20


In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(
    optimizer=Adam(learning_rate=1e-5),  # Lower learning rate!
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history_finetune = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=5,
    callbacks=[early_stop]
)

Epoch 1/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 127s 5s/step - accuracy: 0.3429 - loss: 1.5538 - val_accuracy: 0.5973 - val_loss: 1.1105
Epoch 2/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 118s 5s/step - accuracy: 0.3267 - loss: 1.5478 - val_accuracy: 0.5973 - val_loss: 1.1283
Epoch 3/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 104s 5s/step - accuracy: 0.3897 - loss: 1.4613 - val_accuracy: 0.5705 - val_loss: 1.1509
Epoch 4/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 102s 5s/step - accuracy: 0.3487 - loss: 1.4545 - val_accuracy: 0.5973 - val_loss: 1.1738


In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"\n✅ Fine-Tuned Test Accuracy: {test_acc*100:.2f}%")

153/153 ━━━━━━━━━━━━━━━━━━━━ 16s 104ms/step - accuracy: 0.6837 - loss: 1.0638

✅ Fine-Tuned Test Accuracy: 69.93%


In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# 🔁 Step 1: Get predictions from the test generator
# Get true labels
y_true = test_generator.classes

# Get predicted probabilities
y_pred_probs = model.predict(test_generator, verbose=1)

# Get predicted class indices
y_pred = np.argmax(y_pred_probs, axis=1)

# 🔠 Step 2: Get class labels
class_labels = list(test_generator.class_indices.keys())

# 📊 Step 3: Print classification report
print("\n🔎 Classification Report:\n")
print(classification_report(y_true, y_pred, target_names=class_labels))

# 🔍 Step 4: Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)

# 🎨 Plotting the matrix
plt.figure(figsize=(10, 8))
disp.plot(cmap="Blues", values_format="d")
plt.title("Confusion Matrix")
plt.xticks(rotation=45)
plt.show()
